In [3]:
import pandas as pd
import plotly.express as px

from tqdm import tqdm
tqdm.pandas()

In [4]:
df = pd.read_csv("../../data/s2orc/s2orc_citations_filtered_with_mag_id.csv", index_col=0)
df = df[~df.abstract.isna()]

In [5]:
import spacy

In [6]:
en = spacy.load('en_core_web_sm')

In [7]:
df.doc =df.abstract.progress_apply(en)

100%|██████████| 42756/42756 [57:34<00:00, 12.38it/s]    
/tmp/ipykernel_1133/3270323092.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.doc =df.abstract.progress_apply(en)


In [9]:
import joblib
joblib.dump(df, "../../data/s2orc/s2orc_citations_filtered_with_mag_id_with_spacy.pkl")

['../../data/s2orc/s2orc_citations_filtered_with_mag_id_with_spacy.pkl']

In [12]:
df['lemmas'] = df.doc.apply(lambda doc: [t.lemma_ for t in doc if t.is_alpha if not t.is_stop if not t.is_punct])

In [15]:
df['tokens_len'] = df['lemmas'].str.len()

In [19]:
px.histogram(df['tokens_len'], log_y=True)

In [21]:
topic_model = BERTopic(verbose=True)


2      1993.0
3      2018.0
8      2010.0
9      2018.0
10     2016.0
        ...  
967    2019.0
968    2009.0
971    2016.0
972    2018.0
973    2017.0
Name: year, Length: 42756, dtype: float64

In [38]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore

In [26]:
dictionary = Dictionary(df['lemmas'])

In [37]:
encoded = df['lemmas'].apply(dictionary.doc2bow)

In [42]:
lda = LdaMulticore(encoded, num_topics=10)

In [51]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()


In [54]:
vis = pyLDAvis.gensim_models.prepare(lda, encoded, dictionary=dictionary)


/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of i

In [55]:
vis
#TODO: napisać pętlę znajdującą najlepszą liczbę topiców

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.038837 -0.005567       1        1  14.418248
3     -0.020010 -0.019138       2        1  11.882010
1     -0.016893  0.019722       3        1  10.874018
6      0.003286  0.006239       4        1  10.851576
0      0.013931  0.016869       5        1  10.253248
5      0.011097 -0.022676       6        1   9.965212
7     -0.007607  0.002152       7        1   9.948044
8     -0.008024 -0.002247       8        1   9.704086
4     -0.014469 -0.011281       9        1   6.395218
9     -0.000148  0.015926      10        1   5.708339, topic_info=               Term          Freq         Total Category  logprob  loglift
108           image  25587.000000  25587.000000  Default  30.0000  30.0000
106         feature  40245.000000  40245.000000  Default  29.0000  29.0000
116           model  49682.000000  49682.000000  Default  28.0000  28.0000
115          method  47479.000000  47479.000000  Default  27.0000  27.0000
93   classification  35367.000000  35367.000000  Default  26.0000  26.0000
..              ...           ...           ...      ...      ...      ...
113         machine    953.708186  16705.880855  Topic10  -5.5715   0.0001
82         accuracy    909.128542  21251.046306  Topic10  -5.6193  -0.2884
107            high    812.589901  15723.872658  Topic10  -5.7316  -0.0995
705      classifier    790.043848  19793.220901  Topic10  -5.7597  -0.3578
108           image    774.181495  25587.178307  Topic10  -5.7800  -0.6348

[950 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
17384      1  0.114585    ABE
17384      8  0.687508    ABE
21723      4  0.181029  ACOPF
21723      5  0.724116  ACOPF
21723      6  0.090514  ACOPF
...      ...       ...    ...
20898      1  0.049862  zebra
20898      2  0.099724  zebra
20898      4  0.049862  zebra
20898      6  0.698069  zebra
20898     10  0.049862  zebra

[3540 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 7, 1, 6, 8, 9, 5, 10])